It is useful to cluster the 1B datasets to around 262k - 1M clusters for IVF indexing with Faiss.
However, it is not feasible to do the clustering within the allocated time for indexing. 

Therefore, here we evaluate other options to break down the clustering cost, while getting the same number of clusters.
The model that we use is: Deep1M (1M database vectors), 4096 clusters (which conveniently breaks down to 2^6 * 2^6)


In [1]:
import numpy as np
import faiss
from faiss.contrib import datasets

In [3]:
ds = datasets.DatasetDeep1B(10**6)

In [4]:
xt = ds.get_train(10**5)
d = ds.d
xb = ds.get_database()
xq = ds.get_queries()
gt = ds.get_groundtruth()


In [5]:
sqrt_nlist = 64
nlist = sqrt_nlist**2

# Flat quantizer

Flat quantizer is what we would like to apprach, but it probably too costly. 
We include it here as a topline.
The measure we use is recall of nearest neighbor vs. number of computed distances.

In [22]:
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [23]:
index.train(xt)
index.add(xb)
index.invlists.imbalance_factor()

1.431187283968

In [24]:
stats = faiss.cvar.indexIVF_stats
for nprobe in 1, 4, 16, 64: 
    index.nprobe = nprobe 
    stats.reset()
    D, I = index.search(xq, 100)
    rank = 1
    recall = (I[:, :rank] == gt[:, :1]).sum() / len(xq)
    print(f"nprobe={nprobe} 1-recall @ {rank}: {recall} dis/q={stats.ndis/len(xq):.2f}")

nprobe=1 1-recall @ 1: 0.3745 dis/q=349.15
nprobe=4 1-recall @ 1: 0.6849 dis/q=1344.67
nprobe=16 1-recall @ 1: 0.9004 dis/q=5040.35
nprobe=64 1-recall @ 1: 0.9793 dis/q=18331.49


# IMI quantizer

The IMI quantizer is a cheap way of breaking down the dataset into buckets. It is a PQ2x6 and each PQ code ends in a separate bucket. 

In [25]:
quantizer = faiss.MultiIndexQuantizer(d, 2, int(np.log2(sqrt_nlist)))

In [26]:
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.quantizer_trains_alone = 1

In [27]:
index.train(xt)
index.add(xb)
index.invlists.imbalance_factor()


16.421237645312

In [28]:
stats = faiss.cvar.indexIVF_stats

for nprobe in 1, 4, 16, 64: 
    index.nprobe = nprobe 
    stats.reset()

    D, I = index.search(xq, 100)
    rank = 1
    recall = (I[:, :rank] == gt[:, :1]).sum() / len(xq)
    print(f"nprobe={nprobe} 1-recall @ {rank}: {recall} dis/q={stats.ndis/len(xq):.2f}")

nprobe=1 1-recall @ 1: 0.437 dis/q=3972.32
nprobe=4 1-recall @ 1: 0.6948 dis/q=9210.20
nprobe=16 1-recall @ 1: 0.8656 dis/q=19246.74
nprobe=64 1-recall @ 1: 0.9613 dis/q=41114.89


So way less efficient than the flat quantizer, due to imbalanced inverted lists. TBH, the IMI quantizer usually sets a cap on the number of distances rather than fixing the number of visited buckets. 

# Residual quantizer

This is a 2-level additive quantizer where the first level is trained first, then the second. Since it is an additive quantizer, the top-k centroids can be retrieved efficiently with lookup tables. 

In [33]:
quantizer = faiss.ResidualCoarseQuantizer(d, 2, int(np.log2(sqrt_nlist)))

In [34]:
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.quantizer_trains_alone = 1

In [35]:
index.train(xt)

In [40]:
quantizer.set_beam_factor(-1)

In [41]:
index.add(xb)
index.invlists.imbalance_factor()

3.604173447168

In [42]:
stats = faiss.cvar.indexIVF_stats

for nprobe in 1, 4, 16, 64: 
    index.nprobe = nprobe 
    stats.reset()

    D, I = index.search(xq, 100)
    rank = 1
    recall = (I[:, :rank] == gt[:, :1]).sum() / len(xq)
    print(f"nprobe={nprobe} 1-recall @ {rank}: {recall} dis/q={stats.ndis/len(xq):.2f}")

nprobe=1 1-recall @ 1: 0.3079 dis/q=878.77
nprobe=4 1-recall @ 1: 0.6091 dis/q=3017.90
nprobe=16 1-recall @ 1: 0.8608 dis/q=9996.18
nprobe=64 1-recall @ 1: 0.9685 dis/q=31318.18


Unfortunately still not very good.  

# 2-level tree quantizer

This is a suggestion by Harsha: just cluster to 64 centroids at the first level and train separate clusterings within each bucket.

In [47]:
# 1st level quantizer 

In [48]:
km = faiss.Kmeans(d, sqrt_nlist)

In [49]:
km.train(xt)

9879.4462890625

In [50]:
centroids1 = km.centroids

In [62]:
xt2 = ds.get_train(500_000)

_, assign1 = km.assign(xt2)
bc = np.bincount(assign1)
o = assign1.argsort()

In [64]:
i0 = 0
c2 = []
for c1 in range(sqrt_nlist): 
    print(c1, end="\r", flush=True)
    i1 = i0 + bc[c1]
    subset = o[i0:i1]
    assert np.all(assign1[subset] == c1)
    km = faiss.Kmeans(d, sqrt_nlist)
    xtsub = xt2[subset]
    km.train(xtsub)
    c2.append(km.centroids)
    i0 = i1

Then we just stack the centroids together and forget about the first level clustering. 
In reality with 262k-1M clusters, we'll train a HNSW or NSG index on top.  

In [65]:
centroids12 = np.vstack(c2)

In [66]:
quantizer = faiss.IndexFlatL2(d)
quantizer.add(centroids12)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [68]:
index.add(xb)
index.invlists.imbalance_factor()

1.200742457344

In [69]:
stats = faiss.cvar.indexIVF_stats
for nprobe in 1, 4, 16, 64: 
    index.nprobe = nprobe 
    stats.reset()
    D, I = index.search(xq, 100)
    rank = 1
    recall = (I[:, :rank] == gt[:, :1]).sum() / len(xq)
    print(f"nprobe={nprobe} 1-recall @ {rank}: {recall} dis/q={stats.ndis/len(xq):.2f}")

nprobe=1 1-recall @ 1: 0.3774 dis/q=291.20
nprobe=4 1-recall @ 1: 0.6847 dis/q=1153.03
nprobe=16 1-recall @ 1: 0.8995 dis/q=4459.66
nprobe=64 1-recall @ 1: 0.9825 dis/q=16942.70


Turns out this is very good: same level of accuracy as the flat topline!